In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import logging
from collections import namedtuple
import pickle
import multiprocessing
import time

from ufo_downloader import UFOData, LocationFinder

In [2]:
dataclass = UFOData('data.json')

In [3]:
finder = LocationFinder('location_cache.json')

In [7]:
for i, d in enumerate(dataclass.data):
    try:
        dataclass.data[i].update(finder.find_ugly(d['City'], d['State']))
    except Exception as e:
        print(e)
        break
    if not (i % 100):
        print(i)
finder.save_cache()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
Expecting value: line 1 column 1 (char 0)


In [ ]:
t = time.time()

def get_data_from_table(url:str,generate_desc:bool = False):
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    header = [i.text for i in  soup.thead.findAll("th")]
    data = []
    for row in soup.tbody.findAll("tr"):
        long_desc = ""
        if generate_desc:
            try:
                long_desc =  BeautifulSoup(requests.get(base_page+row.find("a").get("href")).text,"html.parser").tbody.findAll("td")[1].text
            except Exception as e:
                logger.log(level=logging.INFO,msg="could not retrieve long description")
        i = 0
        entry = {}
        for cell in row.findAll("td"):
            entry[header[i]] = cell.text
            i+=1
        entry["Description"] = long_desc
        data.append(entry)
    return data


# Entry = namedtuple("Entry",['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

bs = BeautifulSoup(resp.text,"html.parser")
for link in bs.table.tbody.findAll("a"):
    links_to_parse.append(base_page+link.get('href'))

    
#links_to_parse = ["http://www.nuforc.org/webreports/ndxschanged.html","http://www.nuforc.org/webreports/ndxsCrescent.html","http://www.nuforc.org/webreports/ndxsCross.html","http://www.nuforc.org/webreports/ndxsDelta.html","http://www.nuforc.org/webreports/ndxsDome.html","http://www.nuforc.org/webreports/ndxsFlare.html","http://www.nuforc.org/webreports/ndxsHexagon.html","http://www.nuforc.org/webreports/ndxspyramid.html","http://www.nuforc.org/webreports/ndxsRound.html"]

for link in links_to_parse:
    data = get_data_from_table(link,False)
    parsed_data.extend(data)
    print("done: "+link +" count: "+str(len(data)))
    
print(time.time()-t)
print("checking count, should be around 113241 : "+len(parsed_data))

csv.register_dialect("semicolon",delimiter=";")
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    dw.writerows(parsed_data)


In [ ]:
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    for item in parsed_data :
        try:
            dw.writerow(item)
        except Exception as e:
            print(item)

In [ ]:
with open("big_save.pckl","wb") as f:
    pickle.dump(parsed_data,f)
